# Representing Mutable List

In [191]:
def Pair(x,y):
    def dispatch(msg):
        if msg == "fst":
            return x
        elif msg == "snd":
            return y
        elif msg == "repr":
            return f"Pair({x},{y})"
        elif msg == "print":
            print(dispatch("repr"))
            return dispatch("repr")
        else:
            raise Exception(f"{msg}: Unknown message for Pair instance") 
    return dispatch

In [192]:
def li_str(li):
    if li is not None:
        return f"Pair({li('fst')}, {li_str(li('snd'))})"


def enumerate_li(a, b, stepper = lambda x : x + 1):
    def recur(a):
        if a > b:
            return None
        else:
            return Pair(a, recur(stepper(a)))
    return recur(a)

li = Pair(1, Pair(2, Pair(3, Pair(4, None))))
print(li_str(li))
print(li("fst"))
print(li("snd")("fst"))
print(li("snd")("snd")("snd")("snd"))

print(li_str(enumerate_li(1, 4)))

Pair(1, Pair(2, Pair(3, Pair(4, None))))
1
2
None
Pair(1, Pair(2, Pair(3, Pair(4, None))))


In [193]:
def push(li, x):
    return Pair(x, li)
li = None
li = push(li, 9)
li = push(li, "LISP")
li = push(li, "Yin Yang")
li_str(li)

'Pair(Yin Yang, Pair(LISP, Pair(9, None)))'

It would be nicer that reassignment is not required. To avoid reassignment, we can do something similar in Chapter Message Passing Style.

In [194]:
def Stack():
    li = None
    def push(x):
        nonlocal li
        li = Pair(x, li)
        return li
    return push

li = Stack()
print(li_str(li(7)))
print(li_str(li("LISP")))
print(li_str(li("Yin Yang")))

Pair(7, None)
Pair(LISP, Pair(7, None))
Pair(Yin Yang, Pair(LISP, Pair(7, None)))


## Adding Mutators for `Pair`

In [195]:
def Pair(x,y):
    def dispatch(msg):
        if msg == "fst":
            return x
        elif msg == "snd":
            return y
        elif msg == "set_fst!":
            def f(n):
                nonlocal x
                x = n
            return f
        elif msg == "set_snd!":
            def f(n):
                nonlocal y
                y = n
            return f
        elif msg == "repr":
            return f"Pair({x},{y})"
        elif msg == "print":
            print(dispatch("repr"))
            return dispatch("repr")
        else:
            raise Exception(f"{msg}: Unknown message for Pair instance") 
    return dispatch

In [196]:
li = enumerate_li(0,3)
print(li_str(li))
li("snd")("snd")("set_fst!")("FLAGGED")
print(li_str(li))
li("snd")("set_snd!")(None)
print(li_str(li))

Pair(0, Pair(1, Pair(2, Pair(3, None))))
Pair(0, Pair(1, Pair(FLAGGED, Pair(3, None))))
Pair(0, Pair(1, None))


# Stack, Queue, Deque

Let start with some applications,

In reality, we often familiar with the **queue** which follows the concept of *first come, first serve*. But some prefer to call it **first in first out** (FIFO). For example, it is just nature to use queue to manage orders in ticketing system.

However, **stack** follows the concept of **last in first out**. Often, a pile of plates is taken as analogies to describe how stack works. The most recent plate is at the top, then it will be also the first taken out. 
A more relevant application would be the undo-redo operations in editor applications. They are actually follow the concept of stack (why?). 
Unbeknownst to many, many CPU processor dedicate one of its register to store the pointer ot stack for fast access.

Anyway, this note will provide implementation of queue and stack. You should find out why `enqueue` and `dequeue` are constant time operation.

**Exercise**: Implement Deque, where push, pop, enqueue are constant time operation, including **the case of deleting the Node at the back of the list**.

Tips: Maintain back pointer for each Node

In [197]:
def Queue():
    li = None
    back = None
    def enqueue(x):
        nonlocal back
        new_back = Pair(x, None)
        if back is None:
            nonlocal li
            back = li = new_back
            return li
        else:
            back("set_snd!")(new_back)
            back = new_back
            return li
    def dequeue():
        nonlocal li, back
        li = li("snd")
        back = back if li is not None else back
        return li
    def repr():
        def recur(li):
            if li("snd") is not None:
                return f"{li('fst')}, {recur(li('snd'))}"
            else:
                return li('fst')
        return f"({recur(li)})"
    def dispatch(msg):
        if msg == "enqueue!":
            return enqueue
        elif msg == "dequeue!":
            return dequeue
        elif msg == "repr":
            return repr
        else:
            raise Exception(f"{msg}: Unknown message for Queue instances")
    return dispatch

In [198]:
def Stack():
    li = None
    def push(x):
        nonlocal li
        li = Pair(x, li)
        return li
    def pop():
        nonlocal li
        li = li("snd")
        return li
    def repr():
        def recur(li):
            if li("snd") is not None:
                return f"{li('fst')}, {recur(li('snd'))}"
            else:
                return li('fst')
        return f"({recur(li)})"
    def dispatch(msg):
        if msg == "push!":
            return push
        elif msg == "pop!":
            return pop
        elif msg == "repr":
            return repr
        else:
            raise Exception(f"{msg}: Unknown message for Stack instances")
    return dispatch

In [199]:
s = Stack()
s("push!")("Reisen")
s("push!")("Tewi")
s("push!")("Ringo")
s("push!")("Seiran")
print(s("repr")())
s("pop!")()
print(s("repr")())
s("push!")("Kaguya")
s("push!")("Eirin")
print(s("repr")())

(Seiran, Ringo, Tewi, Reisen)
(Ringo, Tewi, Reisen)
(Eirin, Kaguya, Ringo, Tewi, Reisen)


In [200]:
q = Queue()
q("enqueue!")("Komachi")
q("enqueue!")("Yuyuko")
q("enqueue!")("Yukari")
print(q("repr")())
q("dequeue!")()
print(q("repr")())
q("enqueue!")("Coffin Dancers")
print(q("repr")())

(Komachi, Yuyuko, Yukari)
(Yuyuko, Yukari)
(Yuyuko, Yukari, Coffin Dancers)


# Represent Binary Search Tree

In [203]:
def Node(x, left = None, right = None):
    def dispatch(msg):
        if msg == "data":
            return x
        elif msg == "left":
            return left
        elif msg == "right":
            return right
    return dispatch

def data(node):
    return node("data")

def left(node):
    return node("left")

def right(node):
    return node("right")

def Table():
    # Table is a binary tree that store Node
    # each Node is pair of key, and value
    # then the tree is ordered by key
    tbl = None
    def key(p):
        return p("fst")
    def at(k):
        def recur(node):
            if node is None:
                return None
            elif k < key(data(node)):
                return recur(left(node))
            elif k > key(data(node)):
                return recur(right(node))
            else: # k is in the tree
                return data(node)("snd")
        return recur(tbl)
    def insert(k, v):
        x = Pair(k, v)
        def recur(node):
            if node is None:
                return Node(x)
            elif key(x) < key(data(node)):
                return Node(data(node), recur(left(node)), right(node))
            elif key(x) > key(data(node)):
                return Node(data(node), left(node), recur(right(node)))
            else: # x is in the tree
                return Node(x, left(node), right(node))
        nonlocal tbl
        tbl = recur(tbl)
        return tbl
    def dispatch(key, val = None):
        key = str(key)
        if val is None:
            return at(key)
        else:
            return insert(key, val)
    return dispatch

# Application of `Table`: Work Like `Pyhton`'s `dict`

In [204]:
tbl = Table()
tbl(9, "Cirno")
print(tbl(9))

tbl("Reimu", None)
print(tbl("Reimu"))

info = Table()
info("age", 16)
info("alias", "Black White Human Magician")
print(info("alias"))
tbl("Marisa", info)
print(tbl("Marisa")("alias"), tbl("Marisa")("age"))

alias = Pair("Girl of Knowledge", Pair("Elemental Magician", Pair("Librarian", None)))
tbl("Patchouli", alias)
print(li_str(tbl("Patchouli")))

Cirno
None
Black White Human Magician
Black White Human Magician 16
Pair(Girl of Knowledge, Pair(Elemental Magician, Pair(Librarian, None)))


# Application of `Table` : Represeting Object

In [137]:
def Bank_Acc(nm, amt = 0):
    def withdraw(this, amt):
        this("amt", this("amt") - amt)
    def deposit(this, amt):
        this("amt", this("amt") + amt)
    def repr(this):
        return f"Bank_Acc({this('nm')},{this('amt')})"
    this = Table()
    this("withdraw", withdraw)
    this("nm", nm)
    this("amt", amt)
    this("deposit", deposit)
    this("repr", repr)
    return this

def call_dict_fn(inst, name, *args):
    return inst(name)(inst, *args)

reimu_acc = Bank_Acc("Hakurei Reimu", 100)
call_dict_fn(reimu_acc, "withdraw", 20)
print(call_dict_fn(reimu_acc, "repr"))
call_dict_fn(reimu_acc, "deposit", 120)
print(call_dict_fn(reimu_acc, "repr"))

Bank_Acc(Hakurei Reimu,80)
Bank_Acc(Hakurei Reimu,200)
